In [1]:
import json
import requests
from pprint import pprint
from bs4 import BeautifulSoup as bs

In [2]:
from webview.models import Professor, Subject
from django.conf import settings

In [3]:
#Adding Faculty to database 
def getFacultyList()  :
    """
    Retruns list of dicts : 
    Same DICT : 
        {'department': 'Aerospace Engineering',
         'dept_code': 'AE',
         'designation': 'Assistant Professor',
         'encrypt_key': 'ae-ap',
         'faculty': '<a href="department/AE/faculty/ae-ap">Akshay Prakash</a>'}
    """
    
    url = "http://www.iitkgp.ac.in/facultylist"

    querystring = {"processOn":"onload","colName":"","searchContent":"","_":"1546070278570"}

    headers = {
        'Cookie': "_ga=GA1.3.1618811768.1543517854; JSESSIONID=0830812A57CAE824DAB209654E57B4EE",
        'Accept-Encoding': "gzip, deflate",
        'Accept-Language': "en-IN,en-US;q=0.9,en;q=0.8",
        'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36",
        'Accept': "application/json, text/javascript, */*; q=0.01",
        'Referer': "http://www.iitkgp.ac.in/faclistbydepartment",
        'X-Requested-With': "XMLHttpRequest",
        'Connection': "keep-alive",
        'cache-control': "no-cache",
        'Postman-Token': "6be29381-c68c-40f6-a10d-e63045691718"
        }

    response = requests.request("GET", url, headers=headers, params=querystring)

    return json.loads(response.text)["data"]

def getFacultyUrl(data) : 
    """
    Return faculty name, url 
    Sample Argument : '<a href="department/AE/faculty/ae-ap">Akshay Prakash</a>'
    """
    BASE_URL = "http://www.iitkgp.ac.in/"
    soup = bs(data)
    tag = soup.find("a")
    name = tag.text
    url = BASE_URL + tag["href"]
    return name, url


def addFaculty(name, depCode, url, designation) : 
    data = Professor() 
    data.name = name
    data.departmentCode = depCode
    data.website = url
    data.designation = designation 
    data.save()

In [4]:
facList = getFacultyList()
FacultyDepartmentDict = dict() #This dictionary keeps the mapping of dep code to department name of 

In [6]:
for faculty in facList : 
    name, url = getFacultyUrl(faculty["faculty"])
    FacultyDepartmentDict[faculty["dept_code"]] = faculty["department"]
    addFaculty(name, faculty["dept_code"], url, faculty["designation"])

In [10]:
def addSubject(subCode, subName, credits) : 
    print(subName)
    data = Subject()
    data.code = subCode
    data.name = subName 
    data.credits = credits
    data.save()

In [11]:
#Adding subjects
with open("/Volumes/Data/CODE/test/rateMyProf/result.json", "r") as f : 
    """
    Format : 
        'AE21001': ['INTRODUCTION TO AERODYNAMICS', '4'],
        Sub_CODE : [SUB NAME, Credits]
    """
    subjectList = json.loads(f.read())

In [17]:
for subCode, li in subjectList.items() : 
    addSubject(subCode, li[0], int(li[1]))

ELECTRICAL TECHNOLOGY
ELECTRICAL TECHNOLOGY LAB.
PHYSICS
ENGINEERING DRAWING AND COMPUTER GRAPHICS
CHEMISTRY LAB.
MATHEMATICS-II
ENGLISH FOR COMMUNICATION
EXTRA ACADEMIC ACTIVITY-II
INTRODUCTION TO MANUFACTURING PROCESSES
PROGRAMMING AND DATA STRUCTURE LAB.
MATHEMATICS-I
MECHANICS
CHEMISTRY
PROGRAMMING AND DATA STRUCTURE
EXTRA ACADEMIC ACTIVITY-I
PHYSICS LAB.
FLUID MECHANICS
CROP PRODUCTION TECHNOLOGY
EXTRA ACADEMIC ACTIVITY-III
SCIENCE OF LIVING SYSTEM
PARTIAL DIFFERENTIAL EQUATIONS
ENVIRONMENTAL SCIENCE
SOIL TECHNOLOGY
CROP PRODUCTION TECHNOLOGY LAB.
SOIL TECHNOLOGY LAB.
BASIC ELECTRONICS
THERMODYNAMICS
AGRICULTURAL BIOTECHNOLOGY LAB.
BASIC ELECTRONICS LAB.
PROBABILITY AND STATISTICS
ECONOMICS
EXTRA ACADEMIC ACTIVITY-IV
LAND AND WATER RESOURCES ENGINEERING LAB.
FOOD ENGINEERING LAB.
LAND AND WATER RESOURCES ENGINEERING
AGRICULTURAL MACHINERY
PRINCIPLES OF FOOD ENGINEERING
THEORY AND DESIGN OF MACHINE ELEMENTS
AGRICULTURAL MACHINERY LAB.
IRRIGATION AND DRAINAGE ENGINEERING LABORATORY


POWER SYSTEM PROTECTION
HIGH VOLTAGE DIRECT CURRENT TRANSMISSION (HVDC) AND FLEXIBLE ALTERNATING CURRENT TRANSMISSION SYSTEMS (FACTS)
POWER AND ENERGY SYSTEM SIMULATION LAB
POWER SYSTEM DYNAMICS AND CONTROL
PROJECT - PART 3
RENEWABLE AND DISTRIBUTED ENERGY SYSTEMS
POWER AND ENERGY SYSTEMS LABORATORY
PROJECT - PART 4
COMPREHENSIVE VIVA
DIGITAL VOICE AND PICTURE COMMUNICATION
TELECOMMUNICATION SYSTEMS ENGG. LAB
MODERN DIGITAL COMMUNICATION TECHNIQUES
MOBILE COMMUNICATIONS AND FADING
DESIGN AND SIMULATION LAB-I
TELECOMMUNICATION SWITCHING AND NETWORKS
DIGITAL IMAGE PROCESSING
EMBEDDED SYSTEMS DESIGN
COMPUTER COMMUNICATION AND NETWORKING
IMAGE PROCESSING LABORATORY
PATTERN RECOGNITION AND IMAGE UNDERSTANDING
COMPUTER COMMUNICATION LABORATORY
MINERALOGY
PHYSICS OF SOLID EARTH
INTRODUCTION TO EARTH SCIENCE LAB.
INTRODUCTION TO EARTH SCIENCE
PETROLOGY
PALEONTOLOGY AND  STRATIGRAPHY
DETERMINATIVE MINERALOGY LABORATORY
STRUCTURAL GEOLOGY LAB
FIELD WORK
STRUCTURAL GEOLOGY
MINERAL RESOURCES LAB.


MINE SURVEYING
MINE SURVEYING LABORATORY
MINING MACHINERY
UNDERGROUND COAL MINING
MINING MACHINERY LABORATORY
UNDERGROUND METAL MINING
VENTILATION LABORATORY
VENTILATION AND CLIMATE CONTROL
QUANTITATIVE DECISION MAKING
MINE HAZARDS AND RESCUE LABORATORY
SURFACE MINING
APPLIED ROCK ENGINEERING
ECONOMICS OF MINING ENTERPRISES
FIELD TRIPS
MINE HAZARDS AND RESCUE
INDUSTRIAL TRAINING
ENVIRONMENTAL ENGG. LAB.
ENVIRONMENTAL ENGINEERING
MINE SAFETY AND LEGISLATION
COMPREHENSIVE VIVA-VOCE
ELECTRONICS LAB.
ELECTRONICS FOR PHYSICISTS
ELECTRODYNAMICS
QUANTUM MECHANICS I
MATHEMATICAL METHODS I
ELECTROMAGNETISM AND OPTICS LAB. A
CLASSICAL MECHANICS
COMPUTATIONAL PHYSICS LAB.
COMPREHENSIVE VIVA VOCE-I
QUANTUM MECHANICS-II
NUCLEAR AND PARTICLE PHYSICS LAB I
NUCLEAR AND PARTICLE PHYSICS -I
CONDENSED MATTER PHYSICS-I
MATHEMATICAL METHODS -II
COMPUTATIONAL METHODS
ELECTROMAGNETISM AND OPTICS LAB. B
PROJECT PART I
STATISTICAL PHYSICS-I
NUCLEAR AND PARTICLE PHYSICS LAB-II
CONDENSED MATTER PHYSICS LAB-I
NUC

BIOMATERIALS-TISSUE INTERACTIONS
TECHNOLOGY OF NATURAL AND SYNTHETIC ELASTOMERS
INTRODUCTION TO NANOTECHNOLOGY AND NANO-STRUCTURED MATERIALS
ELECTRON PARAMAGNETIC RESONANCE IN MATERIALS
WATER POWER ENGINEERING
STATISTICAL METHODS IN HYDROLOGY
WATER RESOURCES SYSTEMS
DESIGN OF HYDRAULIC STRUCTURES
ENVIROMMENTAL STATISTICS AND EXPERIMENTAL DESIGN
ENVIRONMENTAL ENGINEERING
ENVIRONMENTAL IMPACT ASSESSMENT AND AUDITING
HAZARDOUS WASTE MANAGEMENT
KNOWLEDGE BASED EXPERT SYSTEMS IN STRUCTURAL ENGG.
THEORY OF ELASTICITY AND PLASTICITY
CONSTRUCTION AND BUILDING MATERIALS
CONSTRUCTION PLANNING AND MANAGEMENT
ADSORPTION AND CATALYSIS
ORGANIC PHOTOCHEMISTRY AND  PERICYCLIC REACTIONS
BIOPHYSICAL CHEMISTRY
ENVIRONMENTAL CHEMISTRY
ADVANCED BIO-INORGANIC CHEMISTRY
METALLOPOLYMERS FOR SMART APPLICATIONS
INORGANIC PHOTOCHEMISTRY
STATISTICAL MECHANICS FOR CHEMISTS
ADVANCED QUANTUM CHEMISTRY
CHEMISTRY OF NANOMATERIALS
ADVANCED STEREOCHEMISTRY
AGROCHEMICALS
REAGENTS IN ORGANIC SYNTHESIS
PHYSICAL ORGANIC CHE

COMMUNICATIONS SIGNAL PROCESSING AND ALGORITHMS
DIGITAL SIGNAL PROCESSING
ADVANCED SENSING TECHNIQUES
MACHINERY FAULT DIAGNOSTICS AND SIGNAL PROCESSING
ADVANCED OPERATING SYSTEM DESIGN
BIOMEDICAL SYSTEM ENGINEERING AND AUTOMATION
DESIGN AND ANALYSIS OF ALGORITHMS
LIGHTWAVE NETWORKS
RF AND MICROWAVE NETWORKS
EMI AND EMC TECHNIQUES
RADAR SIGNATURE ANALYSIS AND RADAR IMAGING
TECHNOLOGY CAD
ADVANCES IN DIGITAL SATELLITE COMMUNICATIONS
LINEAR ALGEBRA AND ERROR CONTROL TECHNIQUES
SECURE COMMUNICATIONS
VISION AND VISUALIZATION
FUNDAMENTALS OF ELECTRONIC PACKAGING
ADVANCED SEMICONDUCTOR DEVICES
ADVANCED MICROWAVE FILTER DESIGN
FIBRE OPTIC SENSORS
NONLINEAR OPTICS
LASER TECHNOLOGY
OPTICAL SIGNAL PROCESSING
SELECTED TOPICS IN FOLE
ANTENNAS AND PROPAGATION FOR WIRELESS COMMUNICATION
FUZZY SET THEORY AND APPLICATIONS
RADAR SIGNAL PROCESSING
SIGNAL PROCESSING / TELECOMMUNICATION ICS
TELEMATICS AND INFORMATICS
SELECTED TOPICS IN TSE
EMBEDDED VLSI SYSTEM
INTEGRATED NANOPHOTONICS
INTELLIGENT GAME DESI

SUBMARINE AND SUBMERSIBLES
MARINE ACOUSTICS
OCEAN ENEGY
NUMERICAL SHIP AND OFFSHORE HYDRODYNAMICS
COASTAL ENGINEERING
DESIGN OF SUBSEA PIPELINES
DESIGN OF INLAND WATERWAYS AND VESSELS
MECHANICAL AND THERMAL BEHAVIOUR OF POLYMERS
AGGLOMERATION AND DIRECT REDUCTION OF IRON ORES
ADVANCED COMPOSITE MATERIALS
ADVANCED CERAMIC TECHNOLOGY
CERAMIC AND REFRACTORIES: SYNTHESIS, PROPERTIES AND APPLICATIONS
SURFACE MODIFICATION AND COATING TECHNOLOGY
SURFACES AND INTERFACES
DIFFRACTION TECHNIQUES IN MATERIALS ENGINEERING
RATE PHENOMENA IN METALLURGICAL PROCESS
SOLIDIFICATION PROCESSING
ADVANCED MECHANICAL BEHAVIOUR OF MATERIALS
COMPUTATIONAL FLIUD FLOW, HEAT AND MASS TRANSFER IN METALLURGICAL PROCESSES
ADVANCED QUANTITATIVE DECISION MAKING
GEODESY AND PHOTOGRAMMETRY
SOFT COMPUTING FOR GEORESOURCES
FUNDAMENTALS AND APPLICATIONS OF GPS TECHNOLOGIES
NUMERICAL METHODS FOR SUBSURFACE ENVIRONMENT
SURFACE MINE DESIGN
FLUID POWER IN MINING MACHINERY
PRINCIPLES OF MINERAL PROCESSING
GROUND CONTROL
COMPUTAT

In [3]:
asd = Subject.objects.all()

In [16]:
a = set()
len("HIGH VOLTAGE DIRECT CURRENT TRANSMISSION (HVDC) AND FLEXIBLE ALTERNATING CURRENT TRANSMISSION SYSTEMS (FACTS)")

109

In [5]:
for t in asd : 
    a.add(t.code[0:2])

In [6]:
pprint(a
     )

{'AE',
 'AG',
 'AR',
 'AT',
 'BM',
 'BS',
 'BT',
 'CE',
 'CH',
 'CR',
 'CS',
 'CY',
 'EA',
 'EC',
 'EE',
 'EG',
 'EP',
 'ES',
 'ET',
 'EV',
 'EX',
 'FP',
 'GG',
 'HS',
 'ID',
 'IE',
 'IM',
 'IP',
 'IT',
 'MA',
 'ME',
 'MF',
 'MI',
 'MM',
 'MS',
 'MT',
 'NA',
 'NT',
 'PH',
 'QE',
 'QM',
 'RE',
 'RT',
 'TE',
 'TS',
 'WM'}


In [22]:
d = {}
for key, val in settings.FACULTY_DEPARTMENT_DICT.items()  : 
    d[val] = key
pprint(d)

{'Advanced Technology Development Centre': 'AT',
 'Aerospace Engineering': 'AE',
 'Agricultural and Food Engineering': 'AG',
 'Architecture and Regional Planning': 'AR',
 'Bio Science': 'BS',
 'Biotechnology': 'BT',
 'Centre For Educational Technology': 'ET',
 'Centre For Oceans,Rivers,Atmosphere and Land Science': 'CL',
 'Centre for Computational and Data Sciences': 'CD',
 'Chemical Engineering': 'CH',
 'Chemistry': 'CY',
 'Civil Engineering': 'CE',
 'Computer Science and Engineering': 'CS',
 'Cryogenic  Engineering': 'CR',
 'Deysarkar Centre of Excellence in Petroleum Engineering': 'DE',
 'Electrical Engineering': 'EE',
 'Electronics and Electrical Communication Engg.': 'EC',
 'Energy Science and Engineering': 'ES',
 'Environmental Science and Engineering': 'EF',
 'G.S Sanyal School of Telecommunication': 'GS',
 'Geology and Geophysics': 'GG',
 'Humanities and Social Sciences': 'HS',
 'Industrial and Systems Engineering': 'IM',
 'Materials Science Centre': 'MS',
 'Mathematics': 'MA',

In [20]:
def getSubjectFromCode(code) : 
    try : 
        sub = Subject.objects.filter(code=code)
        print(len(sub))
        print(str(sub))
    except  : 
        print(1)
getSubjectFromCode("AG40009")

1
<QuerySet [<Subject: AG40009 - CAD AND SIMULATION OF AGRICULTURAL MACHINERY>]>


In [27]:
def getProfessor(name,depCode) : 
    try : 
        prof = Professor.objects.filter(name=name, departmentCode=depCode) 
        print(prof)
        if len(prof) == 1 : 
            print(str(prof[0]))
    except : 
        pass

In [29]:
getProfessor("Amardip Ghosh", "AE")

<QuerySet [<Professor: Professor object (41efe305-8f3f-408f-a912-69ed8db47b89)>]>
Professor object (41efe305-8f3f-408f-a912-69ed8db47b89)
